# Data Analysis

## View some data

In [1]:
from constants import JORDAN_DATASET_FILEPATH, MAESTRO_DATASET_FILEPATH
from data.jordan_dataset import JordanDataset
from data.maestro_dataset import MaestroDataset

id_train_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="train"
)
id_test_dataset = JordanDataset(
    data_dir=JORDAN_DATASET_FILEPATH,
    split="validation"
)
ood_test_dataset = MaestroDataset(
    data_dir=MAESTRO_DATASET_FILEPATH,
    split="test",
)

/data/scratch/joeltjy1/conda_envs/ood-detection/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Loading train split from /scratch/joel/jordan_dataset...
Loaded 4060 samples from train split
Sample keys: ['input_ids', 'labels']
Skipped 490 bad samples
Loading validation split from /scratch/joel/jordan_dataset...
Loaded 84 samples from validation split
Sample keys: ['input_ids', 'labels']
Skipped 0 bad samples
path /scratch/joel/maestrodata/test.txt


In [11]:
import random
import pandas as pd
from utils.process_tokens import get_readable_events

events_by_dataset = {}
for name, dataset in [("id_train", id_train_dataset), ("id_test", id_test_dataset), ("ood_test", ood_test_dataset)]:
    random_idx = random.randint(0, len(dataset) - 1)
    input_ids = dataset[random_idx]['input_ids']
    if hasattr(input_ids, 'tolist'):
        input_ids = input_ids.tolist()
    events = get_readable_events(input_ids)
    events_by_dataset[name] = events

min_events = min(len(events) for events in events_by_dataset.values())
num_rows = min(min_events, 30)

display_keys = ["misc", "onset", "dur", "inst", "pitch", "antic"]
key_mapping = {
    "misc": "special_token",
    "onset": "onset",
    "dur": "duration",
    "inst": "instrument",
    "pitch": "pitch",
    "antic": "anticipated"
}

columns = pd.MultiIndex.from_product([events_by_dataset.keys(), display_keys], names=['dataset', 'key'])

table_data = []
for event_idx in range(num_rows):
    row = []
    for dataset_name in events_by_dataset.keys():
        events = events_by_dataset[dataset_name]
        event = events[event_idx]
        for display_key in display_keys:
            actual_key = key_mapping[display_key]
            value = event.get(actual_key, "")
            if hasattr(value, 'item'):
                value = value.item()
            elif hasattr(value, 'tolist'):
                value = value.tolist()
            row.append(value)
    table_data.append(row)

df = pd.DataFrame(table_data, columns=columns)
df

dataset id_train                               id_test                         \
key         misc onset   dur inst pitch  antic    misc onset   dur inst pitch   
0            AAR                                    AR                          
1                  0.0  0.16    0    B2  False           0.0  0.12    0   G#3   
2                 0.06  0.24    0   D#4  False          0.01  0.09    0   G#4   
3                 0.17  0.07    0   G#4  False          0.05   0.1    0   C#4   
4                 0.26  0.47    0   D#5  False          0.07  0.11    0   C#5   
5                 0.36  0.05    0   G#4  False          0.51   0.1    0    F3   
6                 0.44  0.08    0   D#4  False          0.55  0.11    0   G#3   
7                  0.5  0.05    0   G#4  False          0.56  0.14    0   G#4   
8                 0.54  0.15    0   C#4  False          0.58  0.11    0   C#4   
9                 0.59   0.4    0   C#5  False           0.6  0.14    0   C#5   
10                0.68  0.03    0   G#4  False          0.64  0.14    0   G#5   
11                0.73  0.11    0   D#4  False          0.65  0.08    0    F4   
12                0.81  0.05    0   G#4  False           1.1   0.1    0   G#3   
13                0.86  0.11    0   D#5  False          1.11  0.15    0   C#4   
14                0.94  0.03    0   G#4  False          1.14  0.15    0    F4   
15                0.94   0.8    0    B3  False          1.22   0.2    0   C#6   
16                1.03  0.34    0   C#4  False          1.22  0.11    0   C#5   
17                1.12  0.69    0   D#4  False          1.22  0.18    0   G#5   
18                1.19  0.19    0   A#4  False          1.23   0.1    0   G#4   
19                1.26  0.24    0   D#5  False          1.23  0.12    0    F5   
20                1.33   0.5    0   A#5  False          1.71  0.84    0   C#4   
21                 1.5  0.12    0   A#4  False          1.76  0.57    0    F4   
22                1.59  0.49    0   C#4  False          1.81  0.48    0    F5   
23                1.63  0.36    0   D#5  False          1.82  0.78    0   G#4   
24                1.76  0.96    0   G#5  False          1.87  1.78    0   G#5   
25                1.93  0.49    0   G#4  False          1.93  0.62    0   C#5   
26                2.23  0.14    0    B3  False          1.96  0.85    0    F6   
27                2.49  0.24    0   D#5  False           2.5  0.87    0    F5   
28                 2.7  0.08    0   G#4  False          2.81  0.19    0   C#6   
29                2.96   0.4    0   A#2  False          2.95  0.28    0    F6   

dataset        ood_test                                
key      antic     misc onset   dur inst pitch  antic  
0                    AR                                
1        False            0.0  0.11    0   A#4  False  
2        False           0.17  0.09    0   D#4  False  
3        False           0.32  0.11    0   C#4  False  
4        False           0.32  0.48    0    F5  False  
5        False           0.32  0.06    0   C#5  False  
6        False           0.46  0.17    0   D#4  False  
7        False           0.66  0.18    0    C4  False  
8        False            0.7  0.38    0    C5  False  
9        False            0.7  0.41    0   D#5  False  
10       False           0.98  0.15    0   D#3  False  
11       False           1.14  0.08    0   G#3  False  
12       False           1.27  0.07    0    C4  False  
13       False           1.42  0.05    0   D#4  False  
14       False            1.5  0.07    0   D#3  False  
15       False            1.6  0.07    0   G#3  False  
16       False           1.75  0.07    0    C4  False  
17       False           1.89  0.07    0   D#4  False  
18       False           2.02  0.05    0   D#3  False  
19       False           2.15  0.09    0   G#3  False  
20       False           2.27  0.45    0    C3  False  
21       False           2.29  0.19    0   D#3  False  
22       False           2.43  0.08    0   D#4  False  
23     

## Running all multivariate tests

In [3]:
from data_analysis.mardia import mardia
from data_analysis.royston import royston
from data_analysis.hz import hz
from extract_layers.pooling_functions import pool_mean_std

num_layers = 24

tests = [mardia, royston, hz]

pooling_function = pool_mean_std

## Extract layers

### Sanity check

On an actually multivariate normal distribution, all tests should return a p-value of close to 1.

In [4]:
import numpy as np

X = np.random.randn(4000, 100)
metrics = {}
for test in tests:
    metrics = metrics | test(X)

print(metrics)




  Subsampling to 1000 samples and 100 features for testing
p_value 1.0
{'skewness': np.float64(171153.895), 'kurtosis': np.float64(-0.131), 'skewness_p': 0.824, 'kurtosis_p': 0.896, 'royston_p': 0.965, 'hz_p_value': 1.0}


In [5]:
import os
import numpy as np
import pandas as pd

from constants import SCRATCH_FILEPATH
from tqdm import tqdm


metrics = []

for layer_idx in tqdm(range(num_layers+1), desc="Processing layers"):
    layer_dir = os.path.join(SCRATCH_FILEPATH, pooling_function.__name__, f"layer_{layer_idx}.npy")
    layer_data = np.load(layer_dir)
    print("Processing layer", layer_idx)
    print("Layer data shape:", layer_data.shape)
    layer_metrics = {}
    for test in tests:
        print("Running test", test.__name__)
        layer_metrics = layer_metrics | test(layer_data)
    metrics.append(layer_metrics)

all_metrics = pd.DataFrame(metrics)


Processing layers:   0%|          | 0/25 [00:00<?, ?it/s]

Processing layer 0
Layer data shape: (4060, 2048)
Running test mardia
  Subsampling to 4060 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4060 samples and 100 features for testing


Processing layers:   4%|▍         | 1/25 [00:02<01:03,  2.66s/it]

p_value <0.001
Processing layer 1
Layer data shape: (4060, 2048)
Running test mardia
  Subsampling to 4060 samples and 100 features for testing
Running test royston
  Subsampling to 1000 samples and 100 features for testing
Running test hz
  Subsampling to 4060 samples and 100 features for testing


Processing layers:   8%|▊         | 2/25 [00:05<00:59,  2.61s/it]

p_value <0.001
Processing layer 2
Layer data shape: (4060, 2048)
Running test mardia
  Subsampling to 4060 samples and 100 features for testing


Processing layers:   8%|▊         | 2/25 [00:05<01:00,  2.63s/it]


KeyboardInterrupt: 

In [ ]:
all_metrics

In [ ]:

## optionally save it
all_metrics.to_csv("all_metrics.csv", index=False)